In [1]:
import leidenalg
import scanpy, phate, magic
import numpy as np
import scprep
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import scale
import gspa

/home/aarthivenkat/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
acute = scanpy.read_h5ad('data/acute_tcells.h5ad')
chronic = scanpy.read_h5ad('data/chronic_tcells.h5ad')

In [4]:
adata = scanpy.concat((acute,chronic))
adata.obs['batch'] = adata.obs['batch'].astype('category')

In [5]:
G_without_regression = phate_op.graph.to_pygsp()

In [35]:
data, data_hvgs = scprep.select.highly_variable_genes(acute.to_df(), acute.var_names, percentile=90)
data_hvg = data[data_hvgs]
data_hvg = data_hvg / np.linalg.norm(data_hvg, axis=0)

## Generate GSPA gene embeddings

In [ ]:
data, data_hvgs = scprep.select.highly_variable_genes(adata.to_df(), adata.var_names, percentile=90)
data_hvg = data[data_hvgs]

In [ ]:
gspa_op = gspa.GSPA(graph=phate_op.graph, J=5)
gspa_op.build_diffusion_operator()
gspa_op.build_wavelet_dictionary()

# Embed gene signals from wavelet dictionary
gene_signals = data_hvg.T # embed highly variable genes
gene_ae, gene_pc = gspa_op.get_gene_embeddings(gene_signals)
gene_localization = gspa_op.calculate_localization()
np.savez('results/GSPA_QR_without_regression.npz', signal_embedding=gene_ae,
         localization_score=gene_localization, genes=data_hvgs.values)